#  Random forest binary text classification
### Baseline Codes for Training

In [22]:
###Import Required Libraries

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline 
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score,roc_auc_score,roc_curve,precision_recall_curve,average_precision_score,recall_score,precision_score,f1_score
from sklearn.metrics import homogeneity_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier



import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

## Gather Training and Testing Data
<br>
1. Import Historical Complaints Data from RSDS (Teradata). Coverage Date: June 2019 - December 2019
<br>
2.Teradata_connect intructions can be found on Readme 
<br>
3.Import idenfied BCOP code breaches and merge it with the historical Data set.
4. Retain the relevant variables needed for Modelling. This is CNCRN_DS(X) and Breach_Type(y). 

In [2]:
#via teradata
import teradata_connect

query = """
select A.CNCRN_ID
    ,A.RECVD_D
    ,A.CNCRN_DS
    from (
    select CNCRN_ID, MAX(START_TS) AS MAX_START_TS
    from PRD_RACOE_PI_CNS_VR.VR_CFMS_CNCRN  
    GROUP BY CNCRN_ID
    where  CAST (RECVD_D AS DATE)  between '2019-07-01' and '2019-12-31') as B
    inner join PRD_RACOE_PI_CNS_VR.VR_CFMS_CNCRN  as A on B.CNCRN_ID = A.CNCRN_ID and A.START_TS = B.MAX_START_TS
"""
hist_complaint_df = teradata_connect.read_teradata_query(query)
hist_complaint_df.head(5)

['SQLite', 'SQLite3', 'PostgreSQL ANSI', 'PostgreSQL Unicode']
Using JDBC connect to the Teradata


CNCRN_ID     RECVD_D                                           CNCRN_DS
0  COM-908270  2019-07-19  Case Management- Lending customer requests for...
1  COM-937313  2019-12-11  You are dissatisfied with delay in assessment ...
2  COM-916025  2019-08-26  Customer had called us to confirm cash withdra...
3  COM-936072  2019-12-05  Customer, would like to cancel his Bpay disput...
4  COM-922281  2019-09-26  I believe in protecting and improving our natu...

In [3]:
#workaround since tera is down:
hist_complaint_df = pd.read_excel('Historical_BCOP.xlsx')

hist_complaint_df.shape
hist_complaint_df.head(5)

Unnamed: 0    CNCRN_ID     RECVD_D  \
0           0  COM-904520  2019-07-01   
1           5  COM-904275  2019-07-01   
2          10  COM-923496  2019-10-03   
3          15  COM-904844  2019-07-02   
4          16  COM-934798  2019-11-27   

                                            CNCRN_DS  
0            credit reporting of the overdue account  
1  customer is not happy that he has not received...  
2  Customer required new pin and crn number, no c...  
3  Details of suggestion I have been a customer i...  
4  Customer is dissatisfied with low deposit rate...

In [4]:
#Import BCOP Breaches data
bcop_breaches = pd.read_excel('Code_Breaches_JuneDec2019.xlsx')
bcop_breaches['CNCRN_ID'] = bcop_breaches['CNCRN_ID'].str.strip()
bcop_breaches.head(5)

BCOP_Breach1                    BCOP_Breach2  \
0         43 When we are recovering a debt                             NaN   
1        26 What we will tell and give you  04 Trained and competent staff   
2            05 Protecting confidentiality                             NaN   
3         43 When we are recovering a debt                             NaN   
4  36 Closing any of your banking services                             NaN   

  BCOP_Breach3 BCOP_Breach4    CNCRN_ID  Breaches  
0          NaN          NaN  COM-904302         1  
1          NaN          NaN  COM-904365         1  
2          NaN          NaN  COM-904432         1  
3          NaN          NaN  COM-904483         1  
4          NaN          NaN  COM-904542         1

In [5]:
#run to check how many breaches we have
count= bcop_breaches['Breaches'].value_counts()
count

1    302
Name: Breaches, dtype: int64

In [36]:
#Merge the CFMS Historical Data on the BCOP breaches imported excel sheet
df_Master_File = pd.merge(hist_complaint_df,bcop_breaches[['CNCRN_ID',
                                                 'Breaches']], 
    on='CNCRN_ID', how='left', indicator=True)
df_Master_File.rename(columns={'Breaches' : 'Breach_Type'}, inplace=True)
df_Master_File.head(5)

Unnamed: 0    CNCRN_ID     RECVD_D  \
0           0  COM-904520  2019-07-01   
1           5  COM-904275  2019-07-01   
2          10  COM-923496  2019-10-03   
3          15  COM-904844  2019-07-02   
4          16  COM-934798  2019-11-27   

                                            CNCRN_DS  Breach_Type     _merge  
0            credit reporting of the overdue account          NaN  left_only  
1  customer is not happy that he has not received...          NaN  left_only  
2  Customer required new pin and crn number, no c...          NaN  left_only  
3  Details of suggestion I have been a customer i...          NaN  left_only  
4  Customer is dissatisfied with low deposit rate...          NaN  left_only

In [37]:
#Validate if there are records dropped
count= df_Master_File['_merge'].value_counts()
count

left_only     35362
both            302
right_only        0
Name: _merge, dtype: int64

## Data Cleansing

1. Remove records with missing concern descriptions and short descriptions as well.

In [46]:
#drop records duplicates
df_Master_File = df_Master_File.drop_duplicates(subset = ['CNCRN_ID','CNCRN_DS'])

#drop records with missing CNCRN_DS
df_Master_File = df_Master_File[df_Master_File['CNCRN_DS'] != '-']
df_Master_File = df_Master_File[df_Master_File['CNCRN_DS'].str.split().str.len() >= 2]
df_Master_File = df_Master_File[~df_Master_File["CNCRN_DS"].str.contains('CARE CONFIDENTIAL INFORMATION IN THIS COM - DO NOT RELEASE TO CUSTOMER')] 
df_Master_File = df_Master_File.dropna(subset = ["CNCRN_DS","CNCRN_ID"],how ='any')
df_Master_File = df_Master_File[df_Master_File.Breach_Type != 'TBD']

In [47]:
df_Master_File["Breach_Type"] = np.array(list(map(lambda x: 1 if x== 1 else 0, df_Master_File.Breach_Type )))
count= df_Master_File["Breach_Type"].value_counts()
count

0    34677
1      302
Name: Breach_Type, dtype: int64

## Text Pre-Processing
<br>
1. Remove records with missing concern descriptions. 
<br>
2.In the preprocess_text() method everything is removed except capital and small English letters, which results in single characters that make no sense. For instance, when you remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s".
<br>
Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.
<br>
3. Create X and y variables in preparation for the modelling process and also separate training and testing data

### Why do we need Text Pre-Processing?
Raw data contain numerical value, punctuation,special character etc. 
These value can hamper the performance of model so before applying any text featurization first we need to convert raw data into meaningful data so we do text preprocessing . 
<br>
1.Remove Noisy Data
<br>
2.Tokenization -splitting data into small chunk of words
<br>
3.Normalization - (BOW, TF-IDF, Word2vec,Glove)

In [9]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    # Convert to low case
    sentence = sen.lower()

    return sentence

In [14]:
X = []
sentences = list(df_Master_File["CNCRN_DS"])
for sen in sentences:
    X.append(preprocess_text(sen))


y = df_Master_File['Breach_Type']

In [19]:
# Split your data X and y, into a training and a test set and fit the pipeline onto the training data
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.3, random_state=0)

## Build your RF model 

MODELLING PROPER
Random Forest With Class Weighting

A simple technique for modifying a decision tree for imbalanced classification is to change the weight that each class has when calculating the “impurity” score of a chosen split point.
Impurity measures how mixed the groups of samples are for a given split in the training dataset and is typically measured with Gini or entropy. The calculation can be biased so that a mixture in favor of the minority class is favored, allowing some false positives for the majority class.

This modification of random forest is referred to as Weighted Random Forest.

Another approach to make random forest more suitable for learning from extremely imbalanced data follows the idea of cost sensitive learning. Since the RF classifier tends to be biased towards the majority class, we shall place a heavier penalty on misclassifying the minority class.

1. Basic BOW
2. Tfidf
3. Tfidf + Balanced Class Weighting
Using Class Weight Value of ‘balanced‘ to automatically use the inverse weighting from the training dataset, giving focus to the minority class.

4. Tfidf + Balanced Class Weighting + Bootstrap (true) (GRID)


HYPERPARAMETERS:

n_estimators = number of trees in the foreset<br/>
max_features = max number of features considered for splitting a node<br/>
max_depth = max number of levels in each decision tree<br/>
min_samples_split = min number of data points placed in a node before the node is split<br/>
min_samples_leaf = min number of data points allowed in a leaf node<br/>
bootstrap = method for sampling data points (with or without replacement)<br/>

## What is Pipelining
Pipelining is combining RF with other functions. You can treat the pipeline as if it were a single machine learning model. 

In [24]:
#BASIC RF
text_clf_rf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()),
                         ('classifier', RandomForestClassifier(random_state=5))])
basic_rf= text_clf_rf.fit(X_train, y_train)
predicted= basic_rf.predict(X_test)
np.mean(predicted== y_test)

0.9911377930245855

### Validate model performance using classificarion report and confusion matrix

### True Positive:
Interpretation: You predicted positive and it’s true.
### True Negative:
Interpretation: You predicted negative and it’s true.
### False Positive: (Type 1 Error)
Interpretation: You predicted positive and it’s false.
### False Negative: (Type 2 Error)
Interpretation: You predicted negative and it’s false.

In [25]:
print('1. The F-1 score of the model {}\n'.format(f1_score(y_test, predicted, average='macro')))
print('2. The recall score of the model {}\n'.format(recall_score(y_test, predicted, average='macro')))
print('3. Classification report \n {} \n'.format(classification_report(y_test, predicted)))
print('4. Confusion matrix \n {} \n'.format(confusion_matrix(y_test, predicted)))

1. The F-1 score of the model 0.4977745872218234

2. The recall score of the model 0.4999038738825339

3. Classification report 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     10403
           1       0.00      0.00      0.00        91

    accuracy                           0.99     10494
   macro avg       0.50      0.50      0.50     10494
weighted avg       0.98      0.99      0.99     10494
 

4. Confusion matrix 
 [[10401     2]
 [   91     0]] 



## TFIDF With Class Weight Balancing
class_weight: allows you to place greater emphasis on a class. For example, if the distribution between class 1 and class 2 is heavily imbalanced, the model can treat the two distributions appropriately.
Default is that all weights = 1. Class weights can be specified in a dictionary.
“Balanced” will create class weights that are inversely proportional to class frequencies, giving more weight to individual occurrences of smaller classes.

In [26]:
class_weight ='balanced'

text_bal_rf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()),
                         ('classifier', RandomForestClassifier(class_weight=class_weight, random_state=5))])
#FIT TDIF + CLASS WEIGHT RF
Bal_rf = text_bal_rf.fit(X_train, y_train)
predicted= Bal_rf.predict(X_test)
np.mean(predicted== y_test)

0.9911377930245855

In [27]:
print('1. The F-1 score of the model {}\n'.format(f1_score(y_test, predicted, average='macro')))
print('2. The recall score of the model {}\n'.format(recall_score(y_test, predicted, average='macro')))
print('3. Classification report \n {} \n'.format(classification_report(y_test, predicted)))
print('4. Confusion matrix \n {} \n'.format(confusion_matrix(y_test, predicted)))

1. The F-1 score of the model 0.4977745872218234

2. The recall score of the model 0.4999038738825339

3. Classification report 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     10403
           1       0.00      0.00      0.00        91

    accuracy                           0.99     10494
   macro avg       0.50      0.50      0.50     10494
weighted avg       0.98      0.99      0.99     10494
 

4. Confusion matrix 
 [[10401     2]
 [   91     0]] 



## GRID SEARCH

In [28]:
# Define the parameter sets to test

parameters = {'classifier__bootstrap': [True],
              'classifier__n_estimators': [200, 400, 600, 800, 1000],
              'classifier__max_features': ['auto', 'log2','sqrt'], 
              'classifier__max_depth': [4, 8, 10, 12],
              'classifier__min_samples_leaf': [3, 4, 5],
              'classifier__criterion': ['gini', 'entropy']}

CV = GridSearchCV(text_bal_rf, parameters, cv=3, scoring='recall', n_jobs=-1)
CV = CV.fit(X_train, y_train)

predicted= CV.best_estimator_.predict(X_test)

# To see the best mean score and the params, run the following code

print('Best_Parameter:\n', CV.best_params_)

Best_Parameter:
 {'classifier__bootstrap': True, 'classifier__criterion': 'gini', 'classifier__max_depth': 4, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 5, 'classifier__n_estimators': 200}


In [29]:
# New Model Evaluation metrics 
print('1. The F-1 score of the model {}\n'.format(f1_score(y_test, predicted, average='macro')))
print('2. The recall score of the model {}\n'.format(recall_score(y_test, predicted, average='macro')))
print('3. Classification report \n {} \n'.format(classification_report(y_test, predicted)))
print('4. Confusion matrix \n {} \n'.format(confusion_matrix(y_test, predicted)))

1. The F-1 score of the model 0.5280183419350262

2. The recall score of the model 0.6280843543652349

3. Classification report 
               precision    recall  f1-score   support

           0       0.99      0.95      0.97     10403
           1       0.05      0.31      0.09        91

    accuracy                           0.94     10494
   macro avg       0.52      0.63      0.53     10494
weighted avg       0.99      0.94      0.96     10494
 

4. Confusion matrix 
 [[9867  536]
 [  63   28]] 



## RF - Balance SubSample
The “balanced_subsample” mode is the same as “balanced” except that weights are computed based on the bootstrap sample for every tree grown.

In [30]:
class_weight ='balanced_subsample'

text_clf_rf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()),
                         ('classifier', RandomForestClassifier(class_weight=class_weight, random_state=5))])

In [31]:
# Define the parameter sets to test

parameters = {'classifier__bootstrap': [True],
              'classifier__n_estimators': [200, 400, 600, 800, 1000],
              'classifier__max_features': ['auto', 'log2','sqrt'], 
              'classifier__max_depth': [4, 8, 10, 12],
              'classifier__min_samples_leaf': [3, 4, 5],
              'classifier__criterion': ['gini', 'entropy']}

CV_B = GridSearchCV(text_clf_rf, parameters, cv=3, scoring='recall', n_jobs=-1)
CV_Bal = CV_B.fit(X_train, y_train)

predicted= CV_Bal.best_estimator_.predict(X_test)

# To see the best mean score and the params, run the following code

print('Best_Parameter:\n', CV_Bal.best_params_)

Best_Parameter:
 {'classifier__bootstrap': True, 'classifier__criterion': 'gini', 'classifier__max_depth': 4, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 5, 'classifier__n_estimators': 200}


In [32]:
# New Model Evaluation metrics 
print('1. The F-1 score of the model {}\n'.format(f1_score(y_test, predicted, average='macro')))
print('2. The recall score of the model {}\n'.format(recall_score(y_test, predicted, average='macro')))
print('3. Classification report \n {} \n'.format(classification_report(y_test, predicted)))
print('4. Confusion matrix \n {} \n'.format(confusion_matrix(y_test, predicted)))

1. The F-1 score of the model 0.5224040796436561

2. The recall score of the model 0.6342670594809401

3. Classification report 
               precision    recall  f1-score   support

           0       0.99      0.94      0.97     10403
           1       0.05      0.33      0.08        91

    accuracy                           0.93     10494
   macro avg       0.52      0.63      0.52     10494
weighted avg       0.99      0.93      0.96     10494
 

4. Confusion matrix 
 [[9767  636]
 [  61   30]] 

